In [47]:
from sklearn.datasets import make_classification
import torch 
from sklearn.model_selection import train_test_split 
from sklearn import metrics

In [2]:
class Dataset:
    def __init__(self,data,targets):
        self.data = data
        self.targets = targets 
        pass 
    def __len__(self):
        return self.data.shape[0]
    def __getitem__(self,idx):
        current_sample = self.data[idx,:]
        current_target = self.targets[idx] 
        return {
            'x':torch.tensor(current_sample,dtype=torch.float),
            'y':torch.tensor(current_target,dtype=torch.long)
        }

In [3]:
data,targets = make_classification(n_samples = 1000)
train_data,test_data,train_targets,test_targets = train_test_split(
    data,targets,stratify=targets
)

In [44]:
train_data

array([[ 1.12030407,  1.15004837,  1.21756101, ...,  0.38137646,
        -0.51325419, -0.53294696],
       [-0.07348904, -0.95324949,  1.10220023, ..., -1.21488878,
         1.38240959, -0.38437062],
       [-0.01462729,  0.80129302,  0.33044084, ..., -1.21414139,
        -0.02867313, -0.92843721],
       ...,
       [-0.3778997 , -1.25156142,  0.55682808, ..., -0.7902278 ,
        -0.507496  ,  0.73708696],
       [-0.94891135, -2.05810035, -0.52545324, ...,  0.10660797,
        -1.5077214 , -1.36364591],
       [-0.48816656, -0.40179273,  0.14988661, ..., -0.57739256,
        -2.01802656, -0.8430597 ]])

In [45]:
train_targets

array([1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,
       1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1,
       0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1,
       1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0,
       1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0,
       0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1,
       0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,

In [4]:
train_data.shape
train_targets.shape

(750,)

In [5]:
train_dataset = Dataset(train_data,train_targets)
test_dataset = Dataset(test_data,test_targets)

train_loader = torch.utils.data.DataLoader(
    train_dataset,batch_size = 4)

test_loader = torch.utils.data.DataLoader(
    test_dataset,batch_size = 4)

In [6]:
model = lambda x, w, b: torch.matmul(x,w)+b  # just a function 

In [7]:
train_data.shape

(750, 20)

In [8]:
W = torch.randn(20,1,requires_grad = True)
b = torch.randn(1,requires_grad = True)
learning_rate = 0.001 

In [9]:
W # initial weights 

tensor([[ 1.4352],
        [-1.2655],
        [-0.9921],
        [ 0.0732],
        [-0.4961],
        [ 2.6431],
        [ 0.8998],
        [ 0.2379],
        [-0.9884],
        [ 1.7659],
        [-0.3769],
        [-0.1246],
        [ 2.3644],
        [ 1.2844],
        [ 0.5269],
        [-0.8863],
        [-0.1474],
        [-0.8642],
        [-0.9187],
        [ 1.0975]], requires_grad=True)

In [10]:
b # initial bias 

tensor([-0.3119], requires_grad=True)

In [36]:
for epoch in range(10):
    epoch_loss = 0 
    for data in train_loader:
        xtrain = data['x']
        ytrain = data['y']
        # making the gradient of the W to none for each epochs 
        if W.grad is not None:
            W.grad_zero_()
        
        output = model(xtrain,W,b)
        loss = torch.mean((ytrain.view(-1)-output.view(-1))**2)
        epoch_loss = epoch_loss+loss.item()
        counter = 0 
        # calculating the gradients from autograd 
        loss.backward() 
        
        with torch.no_grad(): # temporarily disable requires grad for Weights and bias
            W = W - learning_rate*W.grad 
            b = b - learning_rate*b.grad 
            
        # regreshing the required grad for W and b 
        W.requires_grad_(True)
        b.requires_grad_(True)
        counter +=1 
    print('epoch = {},loss={}'.format(epoch,loss/counter))

epoch = 0,loss=0.11973459273576736
epoch = 1,loss=0.1197320967912674
epoch = 2,loss=0.11972980201244354
epoch = 3,loss=0.11972777545452118
epoch = 4,loss=0.11972599476575851
epoch = 5,loss=0.11972484737634659
epoch = 6,loss=0.11972354352474213
epoch = 7,loss=0.11972267925739288
epoch = 8,loss=0.11972200870513916
epoch = 9,loss=0.11972139775753021


In [12]:
W  # final weights after traning

tensor([[ 7.3365e-02],
        [ 8.3213e-01],
        [-1.3923e-02],
        [ 1.5217e-04],
        [-1.4097e-02],
        [ 1.3479e+00],
        [ 1.2337e+00],
        [ 7.0434e-01],
        [-3.8483e-03],
        [ 5.1357e-02],
        [ 2.0053e-02],
        [-1.3954e-02],
        [ 5.5177e-02],
        [ 2.9329e-02],
        [-1.7796e-02],
        [-3.3479e-02],
        [-7.5707e-03],
        [-3.7164e-02],
        [-3.2864e-02],
        [-2.7959e-03]], requires_grad=True)

In [18]:
b  # bias after training

tensor([0.4659], requires_grad=True)

In [37]:
W.shape,xtrain.shape  # checking the dimensions 
# four instances are there in xtrain  ==> we got two outputs 
# two hear represents the no of samples in the final epoch 
# ( instance multiplied with random weights for each feature ) 

(torch.Size([20, 1]), torch.Size([2, 20]))

In [15]:
# making predictions on the test data
outputs = []
labels = []
with torch.no_grad():
    for data in test_loader:
        xtest = data['x']
        ytest = data['y']
        
        output = model(xtest,W,b)
        labels.append(ytest) 
        outputs.append(output)

In [46]:
torch.cat(outputs).view(-1),torch.cat(labels).view(-1)

(tensor([ 0.2914,  0.6227,  0.2883,  0.7466,  0.2939,  0.6568,  0.1949,  0.4300,
          0.7986,  0.7202,  0.0422,  0.0274,  0.5017,  0.1513,  1.2566,  0.1933,
          0.0547,  0.4170, -0.2921,  0.8541,  0.3099,  0.2945,  0.5455,  0.4948,
          0.8270,  0.5150,  0.8282,  0.3797,  0.1005,  0.4720,  0.0543,  0.9408,
          0.2154,  0.0179,  0.7617,  0.1655,  0.5674,  0.2801,  0.2001,  0.6335,
         -0.0286,  0.3925,  0.6134,  0.5484, -0.0274,  0.7407, -0.4796,  0.2276,
          0.0014, -0.4138,  1.0513,  0.7227, -0.2328,  0.1688,  0.0553,  0.6939,
          0.4036,  0.4616,  0.3137,  0.4772,  0.2093,  0.8935,  0.4533,  0.7997,
         -0.0215,  0.2858,  0.3285,  0.5770,  0.8773,  0.6491,  1.2390,  0.2769,
          1.0550,  0.7820,  1.2788,  0.7416,  1.0069,  0.3476,  0.1344, -0.3290,
          1.1308,  0.8306,  0.3426,  0.1804, -0.3826,  0.8540,  0.7587,  1.0753,
         -0.0478,  0.7505,  0.0383,  0.4375,  0.1368,  1.1615,  0.6789,  0.2199,
          0.8149,  0.2379,  

In [49]:
metrics.roc_auc_score(torch.cat(labels).view(-1),torch.cat(outputs).view(-1))

0.987776